# SK Functions, Native Functions and Pipes



## Load the required .NET Packages

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"
#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: dotenv.net"

using System.ComponentModel;
using System.Net.Http;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.AI.ImageGeneration;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.ChatCompletion;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Utils/ConsoleLogger.cs

## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("DAVINCI_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

## Get a kernel instance

In [ ]:
IKernel kernel = new KernelBuilder()
    .WithLogger(ConsoleLogger.Log)
    .WithAzureTextCompletionService(deploymentName,endpoint, apiKey)
    .Build();

## Simple pipeline

In [ ]:
Console.WriteLine("======== Pipeline ========");

// Load native skills
var text = kernel.ImportSkill(new TextSkill());

// Create a new Skill
class ConsoleSkill
{
    [SKFunction, Description("Trim whitespace from the start and end of a string.")]
    public string Log(string input)
    {
        Console.WriteLine(input);
        return input;
    }
}
var cskill = kernel.ImportSkill(new ConsoleSkill());

### Use skills in a pipeline

In [ ]:
SKContext result = await kernel.RunAsync("    i n f i n i t e     s p a c e     ",
    cskill["Log"],
    text["TrimStart"],
    cskill["Log"],
    text["TrimEnd"],
    cskill["Log"],
    text["Uppercase"],
    cskill["Log"]);

## LLM Pipeline

### Create an in-line function to give funny excuses

In [ ]:
const string FunctionDefinition = @"
Generate a creative reason or excuse for the given event.
Be creative and be funny. Let your imagination run wild.

Event: I am running late.
Excuse: I was being held ransom by giraffe gangsters.

Event: I haven't been to the gym for a year
Excuse: I've been too busy training my pet dragon.

Event: {{$input}}
";

var excuseFunction = kernel.CreateSemanticFunction(FunctionDefinition, maxTokens: 100, temperature: 0.4, topP: 1);

### Execute the excuses plugin

In [ ]:
var result = await excuseFunction.InvokeAsync("I missed the F1 final race");
Console.WriteLine(result);

### Pipe the excuses plugin and uppercase the output

In [ ]:
SKContext result1 = await kernel.RunAsync("I missed your birthday",
    excuseFunction,
    text["Uppercase"]);

Console.WriteLine(result1);

### Import a function to summarize

In [ ]:
var sumSkill = kernel.ImportSemanticSkillFromDirectory(
            "./skills",
            "SummarizeSkill");

### Execute a prompt, Summarize the output and uppercase it

In [ ]:
var ask = "What's the tallest building in South America?";

var result2 = await kernel.RunAsync(
    ask,
    sumSkill["Summarize"],
    text["Uppercase"]
);

Console.WriteLine(result2);